In [1]:
import os
import sys
from scipy.special import ndtri
import numpy as np
from matplotlib import pyplot as plt
import corner
# import Nestle
import nestle
#dynesty
import dynesty

%matplotlib inline

In [2]:
"""
Setup a model and data
"""


# set the true values of the model parameters for creating the data
m = 3.5 # gradient of the line
c = 1.2 # y-intercept of the line

# set the "predictor variable"/abscissa
M = 1000
xmin = 0.
xmax = 10.
stepsize = (xmax-xmin)/M
x = np.arange(xmin, xmax, stepsize)

# define the model function
def theory(x, m, c):
    
    return m*x + c

# create the data - the model plus Gaussian noise
sigma = 0.05 # standard deviation of the noise
data = theory(x, m, c) + sigma*np.random.randn(M)


LN2PI = np.log(2.*np.pi)
LNSIGMA = np.log(sigma)

In [3]:
def prior_transform(theta):
    
    bounds = [[0, 10], [-2, 6]]
    priors = []
    # When theta 0-> append bound[0], if theta 1-> append bound[1]
    for c, bound in enumerate(bounds):
        priors.append(theta[c]*(bound[1]-bound[0])+bound[0])

    # At this moment, np.array(priors) has shape (dims,)
    # print("Prior transform : {}".format(np.array(priors)))
    return np.array(priors)

In [4]:
def loglikelihood(theta):
    
    m, c = theta  # unpack the parameters

    chisq = np.sum(((data-theory(x, m, c))/sigma)**2)

    return -0.5*chisq

In [ ]:
bounds = [[0, 10], [-2, 6]]
nlive = 1024     # number of live points
method = 'classic' # use MutliNest algorithm
ndims = 2        # two parameters
tol= 0.5         # the stopping criterion (this is the nestle default, so doesn't need to be set)

res = nestle.sample(loglikelihood, prior_transform, ndims, method=method, npoints=nlive, dlogz=tol)

logZnestle = res.logz                         # value of logZ
infogainnestle = res.h                        # value of the information gain in nats
logZerrnestle = np.sqrt(infogainnestle/nlive) # estimate of the statistcal uncertainty on logZ

# output marginal likelihood
print('Marginalised evidence is {} ± {}'.format(logZnestle, logZerrnestle))

# re-scale weights to have a maximum of one
nweights = res.weights/np.max(res.weights)

postsamples = res.samples

print('Number of posterior samples is {}'.format(postsamples.shape[0]))


In [ ]:
nlive = 1024      # number of (initial) live points
bound = 'multi'   # use MutliNest algorithm
sample = 'rwalk'  # use the random walk to draw new samples

sampler2 = dynesty.NestedSampler(loglikelihood, prior_transform, ndims,
                        bound=bound, sample=sample, nlive=nlive)
sampler2.run_nested(dlogz=0.01)

res2 = sampler2.results

logZdynesty = res2.logz[-1]        # value of logZ
logZerrdynesty = res2.logzerr[-1]  # estimate of the statistcal uncertainty on logZ

# output marginal likelihood
print('Marginalised evidence (using static sampler) is {} ± {}'.format(logZdynesty, logZerrdynesty))

# get the posterior samples
weights = np.exp(res2['logwt'] - res2['logz'][-1])
#postsamples2 = dynesty.utils.resample_equal(res2.samples, weights)
postsamples2 = res2.samples

###fig = corner.corner(dpostsamples, labels=[r"$m$", r"$c$"], truths=[m, c], hist_kwargs={'density': True})
#fig = corner.corner(postsamples, fig=fig, color='r', hist_kwargs={'density': True})
#plt.show()


In [ ]:
print("logz : {} +/- {} ".format(round(res2.logz[-1], 3), round(res2.logzerr[-1], 3)))
print("logLike : {} ".format(round(res2.logl[-1], 3)))

In [ ]:
bounds = [[0, 10], [-2, 6]]

def priorTransform(theta, bounds):
    """
    This is a common prior transform (flat priors).

    Parameters:
        theta  : is a random vector with de dimensionality of the model.
        bounds : list of lists of lower and higher bound for each parameter.
    """
    priors = []
    # When theta 0-> append bound[0], if theta 1-> append bound[1]
    for c, bound in enumerate(bounds):
        priors.append(theta[c]*(bound[1]-bound[0])+bound[0])

    # At this moment, np.array(priors) has shape (dims,)
    # print("Prior transform : {}".format(np.array(priors)))
    return np.array(priors)


In [ ]:
from SkillingNS import SkillingNS

In [ ]:
bounds = [[-2,6], [0,10]]
sampler = SkillingNS(loglikelihood, priorTransform, ndims, bounds,
                     nlivepoints=nlive)
samples = sampler.sampler(accuracy=0.01)


In [ ]:
# postsamples3 = samples['samples']['points'].values
postsamples3 = samples['samples']['points'].to_numpy()
#postsamples3 = postsamples3.ravel()
#np.array(postsamples3).flatten()
#np.array(postsamples3).resize(2,10000)
# rows=int(np.shape(postsamples3)[0]/2)
# ps3 = postsamples3.reshape(5512,2)
np.array(postsamples3)

In [ ]:
type(postsamples)

In [ ]:
fig = corner.corner(postsamples2, labels=[r"$m$", r"$c$"], color = 'k', truths=[m, c])
#fig = corner.corner(postsamples, fig=fig, color='r', hist_kwargs={'density': True})
fig2 = corner.corner(postsamples, labels=[r"$m$", r"$c$"], color = 'b', truths=[m, c])
fig3 = corner.corner(postsamples3, labels=[r"$m$", r"$c$"], color = 'r', truths=[m, c])

